In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

In [10]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [11]:
Family_list=df_train.family.unique()
Family_list

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [12]:
store_nbr_list=df_train.store_nbr.unique()
store_nbr_list

array([ 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,  2, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29,  3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,  4,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  5, 50, 51, 52, 53, 54,  6,
        7,  8,  9])

In [13]:
df_train['date'] = pd.to_datetime(df_train.date,format='%Y-%m-%d')
df_train.index = df_train['date']

df_test['date'] = pd.to_datetime(df_test.date,format='%Y-%m-%d')
df_test.index = df_test['date']

In [14]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
date,,,,,,
2013-01-01,0,2013-01-01,1,AUTOMOTIVE,0.0,0
2013-01-01,1,2013-01-01,1,BABY CARE,0.0,0
2013-01-01,2,2013-01-01,1,BEAUTY,0.0,0
2013-01-01,3,2013-01-01,1,BEVERAGES,0.0,0
2013-01-01,4,2013-01-01,1,BOOKS,0.0,0


In [28]:
!pip install pmdarima
from pmdarima.arima import auto_arima

In [39]:
Flag=1
store_nbr_txt= open("store_nbr.txt","w+")
Family_list_txt= open("Family_list.txt","w+")
Iterations=len(store_nbr_list) * len(Family_list)

for s in range(len(store_nbr_list)):
  for f in range(len(Family_list)):
    training=df_train[df_train['store_nbr']==store_nbr_list[s]]
    training=training[training['family']==Family_list[f]]
    print("***********************************************************************************************")
    print(((Flag*100)/Iterations),'% calculation completed')
    print("Training for", Family_list[f],'in store number',store_nbr_list[s], 'with number of observations',len(training))
    print("***********************************************************************************************")
    
    training_ip=training.drop(['id','date','store_nbr','family','onpromotion'],axis=1)

    model = auto_arima(training_ip,start_p=1, start_q=1,max_p=3, max_q=3, m=12,start_P=0, seasonal=True,d=1, D=1, trace=True,error_action='ignore',suppress_warnings=True)
    model.fit(training_ip)

    testing=df_test[df_test['family']==Family_list[f]]
    testing=testing[testing['store_nbr']==store_nbr_list[s]]

    Predictions=model.predict(n_periods=len(testing))
    forecast = pd.DataFrame(Predictions,index = testing.id,columns=['sales'])

    forecast.reset_index(inplace=True)

    if Flag==1:
      Final_op=forecast
    else:
      Final_op = pd.concat([Final_op,forecast])

    print('Length of Final_op is:', len(Final_op))
    print(Final_op)
    print("#####################################################################################")

    Flag+=1

    store_nbr_txt.write(str(store_nbr_list[s]))
    Family_list_txt.write(Family_list[f])


    del training,training_ip,model,testing,Predictions,forecast
    Final_op.to_csv('ARIMA_CSV.csv', index=False)
    
    
store_nbr_txt.close()
Family_list_txt.close()

***********************************************************************************************
Training for AUTOMOTIVE in store number 1 with number of observations 1684
***********************************************************************************************
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=12.53 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=10404.300, Time=0.09 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=9338.029, Time=0.60 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=7.66 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=10034.606, Time=0.17 sec
 ARIMA(1,1,0)(2,1,0)[12]             : AIC=9172.562, Time=1.52 sec
 ARIMA(1,1,0)(2,1,1)[12]             : AIC=inf, Time=19.57 sec
 ARIMA(1,1,0)(1,1,1)[12]             : AIC=inf, Time=4.27 sec
 ARIMA(0,1,0)(2,1,0)[12]             : AIC=9606.522, Time=1.24 sec
 ARIMA(2,1,0)(2,1,0)[12]             : AIC=8971.048, Time=2.79 sec
 ARIMA(2,1,0)(1,1,0)[12]            

/usr/local/lib/python3.7/dist-packages/pmdarima/arima/auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


Length of Final_op is: 32
         id     sales
0   3000888  2.392053
1   3002670  2.327342
2   3004452  5.836497
3   3006234  1.921794
4   3008016  2.540296
5   3009798  2.637630
6   3011580  2.196435
7   3013362 -0.751594
8   3015144  4.565983
9   3016926  1.003763
10  3018708  2.752044
11  3020490  2.524122
12  3022272  1.126285
13  3024054  1.391966
14  3025836  4.843125
15  3027618  1.327348
0   3000889  0.000000
1   3002671  0.000000
2   3004453  0.000000
3   3006235  0.000000
4   3008017  0.000000
5   3009799  0.000000
6   3011581  0.000000
7   3013363  0.000000
8   3015145  0.000000
9   3016927  0.000000
10  3018709  0.000000
11  3020491  0.000000
12  3022273  0.000000
13  3024055  0.000000
14  3025837  0.000000
15  3027619  0.000000
#####################################################################################
***********************************************************************************************
Training for BEAUTY in store number 1 with number of observations 

/usr/local/lib/python3.7/dist-packages/pmdarima/arima/auto.py:460: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


Length of Final_op is: 608
         id     sales
0   3000888  2.392053
1   3002670  2.327342
2   3004452  5.836497
3   3006234  1.921794
4   3008016  2.540296
..      ...       ...
11  3020527  0.000000
12  3022309  0.000000
13  3024091  0.000000
14  3025873  0.000000
15  3027655  0.000000

[608 rows x 2 columns]
#####################################################################################
***********************************************************************************************
Training for BREAD/BAKERY in store number 10 with number of observations 1684
***********************************************************************************************
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,1,1)[12]             : AIC=inf, Time=6.40 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=18519.090, Time=0.07 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=17955.582, Time=0.89 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=inf, Time=5.10 sec
 ARIMA(1,1,0)(0,1,0)[12]  

KeyboardInterrupt: ignored

In [ ]:
# print(model.summary())

In [40]:
ARIMA_CSV = pd.DataFrame({'id':Final_op['id'], 'sales':Final_op['sales']})
ARIMA_CSV.to_csv('ARIMA_CSV.csv', index=False)